<a href="https://colab.research.google.com/github/Xotik105/VOSK/blob/main/VOSK_speech_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://git.ffmpeg.org/ffmpeg.git ffmpeg

Cloning into 'ffmpeg'...
remote: Enumerating objects: 4407, done.
remote: Counting objects: 100% (4407/4407), done.
remote: Compressing objects: 100% (3210/3210), done.
remote: Total 664960 (delta 3228), reused 1316 (delta 1186)
Receiving objects: 100% (664960/664960), 159.27 MiB | 17.46 MiB/s, done.
Resolving deltas: 100% (537112/537112), done.


In [2]:
# Install dependencies
!pip install wget pydub wave tqdm vosk
!apt-get ffmpeg

# Download Vosk model
!mkdir models
!wget -P models/ https://alphacephei.com/vosk/models/vosk-model-en-us-0.22.zip
!unzip models/vosk-model-en-us-0.22.zip -d models/ && rm models/vosk-model-en-us-0.22.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.2 MB 7.7 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=d6bd2afa1765bc90848688356638b7c4fb4138eed52719d6ca7eca2d9a503077
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
  Created wheel for wave: filename=Wave-0.0.2-py3-none-any.whl size=1240 sha256=d5b32ea2cd1461e188a24025d5d044fb69b0435ecdf8195721d0869bae82b4ca
  Stored in directory: /root/.cache/pip/wheels/25/e8/fe/458c7dac00c6abedad6380b9d0ef1a5cbc7c21807df1d30915
  Created wheel for srt: filename=srt-3.5.2-py3-none-any.whl size=22487 sha256=1dda310874f01e7bb0dacc254cff6c7c19928e33ce443b8e88b4d9466d1fefef
  Stored in directory: /root/.cache/pip/wheels/54/c4/ec/4604122e072aebb16803c8297b7cd3f4c72073a3ee58738015
Successfully built wget wave srt
E: Invalid operation ffmpeg
--2022-06-05 18:34:48--  https

In [1]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
from pydub import AudioSegment
import os

def mp3_to_wav(source, skip=0, excerpt=False):
    
    sound = AudioSegment.from_mp3(source) # load source
    sound = sound.set_channels(1) # mono
    sound = sound.set_frame_rate(16000) # 16000Hz
    
    if excerpt:
        excrept = sound[skip*1000:skip*1000+60000] # 30 seconds - Does not work anymore when using skip
        output_path = os.path.splitext(source)[0]+"_excerpt.wav"
        excrept.export(output_path, format="wav")
    else:
        audio = sound[skip*1000:]
        output_path = os.path.splitext(source)[0]+".wav"
        audio.export(output_path, format="wav")
    
    return output_path

In [4]:
!pip install vosk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
mp3_to_wav('/content/Test_podcat_file.mp3', 0, True)

CouldntDecodeError: ignored

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
from vosk import Model, KaldiRecognizer, SetLogLevel
from tqdm.notebook import tqdm
import wave
import os
import json

def transcript_file(input_file, model_path):
    
    # Check if file exists
    if not os.path.isfile(input_file):
        raise FileNotFoundError(os.path.basename(input_file) + " not found")    
    
    # Check if model path exists
    if not os.path.exists(model_path):
        raise FileNotFoundError(os.path.basename(model_path) + " not found")

    # open audio file
    wf = wave.open(input_file, "rb")
    
    # check if wave file has the right properties
    if wf.getnchannels() != 1 or wf.getsampwidth() != 2 or wf.getcomptype() != "NONE":
        raise TypeError("Audio file must be WAV format mono PCM.")
    
    # Initialize model
    model = Model(model_path)
    rec = KaldiRecognizer(model, wf.getframerate())
    
    # Get file size (to calculate progress bar)
    file_size = os.path.getsize(input_file)
    
    # Run transcription
    pbar = tqdm(total=file_size)

    # To store our results
    transcription = []

    while True:
        data = wf.readframes(4000) # use buffer of 4000
        pbar.update(len(data))
        if len(data) == 0:
            pbar.set_description("Transcription finished")
            break
        if rec.AcceptWaveform(data):
            # Convert json output to dict
            result_dict = json.loads(rec.Result())
            # Extract text values and append them to transcription list
            transcription.append(result_dict.get("text", ""))

    # Get final bits of audio and flush the pipeline
    final_result = json.loads(rec.FinalResult())
    transcription.append(final_result.get("text", ""))
    
    transcription_text = ' '.join(transcription)
    
    return transcription_text

In [6]:
transcription = transcript_file('/content/Recording1.wav', '/content/models/vosk-model-en-us-0.22')

Error: ignored

In [4]:
transcription

"hey everybody welcome to the podcast my yesterday's dr andrew huber man andrew is a neuroscientist he is a neurobiology professor at stanford medical school and mcknight foundation and pew foundation fellow and the founder of humor min lab where he is involved in all kinds of really amazing breakthrough research on brain function brain plasticity and brain regeneration his work has been published in top journals like nature he's been featured in publications like time scientific american the bbc and he's here today to discuss the brain to discuss growth mindset how to focus how to navigate the stressful times and many other subjects it's an incredible conversation i think you guys are going to enjoy it i appreciate you watching make sure to hit that subscribe button and without further ado this is me and dr andrew huber meant and"